In [1]:
import pycromanager
from pycromanager import Core
import numpy as np
import matplotlib.pyplot as plt

core = Core()


In [2]:
def get_data():
    tagged_image = core.get_tagged_image()
    # get the pixels in numpy array and reshape it according to its height and width
    image_array = np.reshape(
        tagged_image.pix,
        newshape=[-1, tagged_image.tags["Height"], tagged_image.tags["Width"]],
    )
    # for display, we can scale the image into the range of 0~255
    image_array = (image_array / image_array.max() * 255).astype("uint8")
    # return the first channel if multiple exists
    return image_array[0, :, :]

In [28]:
import torch

class OnTheFlySegmentation:
    def __init__(self, model, frames: int):
        self.model = model
        self.n_frames = frames
        self.frame_buffer: list[np.ndarray] = []

    def set_n_frames(self, n: int):
        self.n_frames = n
    
    def clear_buffer(self):
        del self.frame_buffer
        self.frame_buffer = []

    def add_image(self, image: np.ndarray):
        self.frame_buffer.append(image)

    def inference(self) -> torch.Tensor:
        if (len(self.frame_buffer) < self.n_frames):
            raise ValueError("Number of frames in buffer smaller than requested amount.")
        
        input_frames = torch.zeros((self.n_frames, *self.frame_buffer[0].shape))
        for i in range(self.n_frames):
            image = input_frames[i]
            input_frames[i, :, :] = image
        self.clear_buffer()

        input_frames = input_frames.unsqueeze(1).unsqueeze(0)

        with torch.no_grad():
            print(f'Input dimensions: {input_frames.shape}')
            result = self.model.forward(input_frames, inference=True)
        return result
        


In [29]:
from models import *

model_class = PLSegmentationUnetScalingModel
model_path = 'saved_models/test_upscale_v5.model'

model = model_class.load(model_path)

Model loaded from: 'saved_models\test_upscale_v5.model'


In [30]:
import tkinter as tk
import numpy as np
from PIL import Image, ImageTk
from pycromanager import Acquisition
from pycromanager import multi_d_acquisition_events

# Create the main window
root = tk.Tk()
root.title("Test On the Fly segmentation")
root.geometry("800x600")

# Create frame for button bar
button_frame = tk.Frame(root, bg="#f0f0f0")
button_frame.pack(side=tk.TOP, fill=tk.X, padx=10, pady=10)

# Function to generate and display a random image
def display_random_image():
    # Generate a random image using NumPy
    width, height = 400, 300
    # Create random RGB data
    img_data = np.random.randint(0, 255, (height, width, 3), dtype=np.uint8)
    
    # Convert NumPy array to PIL Image
    img = Image.fromarray(img_data)
    
    # Convert to Tkinter-compatible image
    photo = ImageTk.PhotoImage(img)
    
    # Update the label with the new image
#    image_label.config(image=photo)
    image_label.config(image=photo)
    image_label.image = photo  # Keep a reference to prevent garbage collection


# Button stubs
def open_image():
    # Stub function for opening an image
    print("Open image button clicked")

def save_image():
    # Stub function for saving an image
    print("Save image button clicked")

def generate_random_image():
    # Generate a new random image
    print("Generating random image")
    display_random_image()

def generate_inference():

    n_frames = int(num_frames_entry.get())
    segmentation = OnTheFlySegmentation(model, n_frames)

    def add_image_func(im_data, metadata, *args):
        segmentation.add_image(im_data)
        return (im_data, metadata)

    print('Starting acquisition')
    with Acquisition(directory=None, name=None, image_process_fn=add_image_func, show_display=False) as acq:
        events = multi_d_acquisition_events(num_time_points=n_frames)
        acq.acquire(events)
    print('Finished acquisition')

    print('Starting inference')
    result = segmentation.inference()
    print('Finished inference')

    pill_img = Image.fromarray(result)
    
    photo = ImageTk.PhotoImage(pill_img)
    image_label.config(image=photo)
    image_label.image = photo
    
    print('\n\n')
# Add buttons to the button bar
open_button = tk.Button(button_frame, text="Open Image", command=open_image)
open_button.pack(side=tk.LEFT, padx=5)

save_button = tk.Button(button_frame, text="Save Image", command=save_image)
save_button.pack(side=tk.LEFT, padx=5)

generate_button = tk.Button(button_frame, text="Generate Random", command=generate_random_image)
generate_button.pack(side=tk.LEFT, padx=5)


entry_label = tk.Label(button_frame, text="Number of frames: ")
entry_label.pack(side=tk.LEFT, padx=5)

num_frames_entry = tk.Entry(button_frame)
num_frames_entry.pack(side=tk.LEFT, padx=2)
num_frames_entry.insert(0, "100")

acquire_button = tk.Button(button_frame, text="Acquire", command=generate_inference)
acquire_button.pack(side=tk.LEFT, padx=2)


# Create frame for image display
image_frame = tk.Frame(root, bg="#ffffff")
image_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=10)

# Label to display image
image_label = tk.Label(image_frame, bg="#ffffff", text="Loading image...")
image_label.pack(fill=tk.BOTH, expand=True)

# Schedule the initial image display after the main loop starts
root.after(100, display_random_image)

# Start the main loop
root.mainloop()

Starting acquisition
Finished acquisition
Starting inference
Input dimensions: torch.Size([1, 100, 1, 512, 512])
Finished inference


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\arnev\AppData\Local\Programs\Python\Python313\Lib\tkinter\__init__.py", line 2068, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "C:\Users\arnev\AppData\Local\Temp\ipykernel_12936\2234527359.py", line 68, in generate_inference
    pill_img = Image.fromarray(result)
  File "c:\Users\arnev\Documents\thesis\CLIM-DL\venv\Lib\site-packages\PIL\Image.py", line 3300, in fromarray
    arr = obj.__array_interface__
          ^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Tensor' object has no attribute '__array_interface__'
